# Prompt_Length-and-Complexity_Management

Effective prompt engineering often requires finding the right balance between providing enough context for the model to understand the task and keeping prompts concise for efficiency. Additionally, many real-world applications involve processing long documents or complex multi-step tasks, which can exceed the context window of LLMs. Learning to manage these challenges is crucial for building robust AI applications.

## Key Components

1. Balancing detail and conciseness in prompts
2. Strategies for handling long contexts
3. Practical examples using OpenAI's GPT model and LangChain

In [1]:
! pip install langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

os.environ['GOOGLE_API_KEY']=''

# Inatiate the LLM
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

def get_completion(prompt):
  """ Helper function to get model completion."""
  return llm.invoke(prompt).content

# Balencing Details and Conciseness

In [3]:
# Detailed prompt
detailed_prompt=PromptTemplate(
    input_variables=['topic'],
    template ="""
          Please provide comprehensive explanation of {topic} include its defination.
          historical context, key components, pratical applications and any relevant example.
          Also discuss controversies and debates around topic and mention potential and future debelopement or trends
    """

)

# Concise prompt
concise_prompt=PromptTemplate(
    input_variables=['topic'],
    template="""
        Briefly explain the {topic} and its main importance
    """
)

topic='Artificial Intelligence'

print('Detailed response:')
print(llm.invoke(detailed_prompt.format(topic=topic)).content)

print('\n Concise response:')
print(llm.invoke(concise_prompt.format(topic=topic)))

Detailed response:
## Artificial Intelligence: A Comprehensive Overview

**Definition:**

Artificial intelligence (AI) is a broad field encompassing the theory and development of computer systems able to perform tasks that normally require human intelligence.  These tasks include learning, reasoning, problem-solving, perception, and natural language understanding.  It's important to distinguish between narrow or weak AI (designed for specific tasks) and general or strong AI (hypothetical AI with human-level intelligence and adaptability across diverse domains).  Currently, most AI systems fall under the category of narrow AI.

**Historical Context:**

The conceptual roots of AI can be traced back to ancient myths of artificial beings endowed with intelligence.  However, the formal field emerged in the mid-20th century:

* **1950s:** Alan Turing's seminal work on computing and his "Turing Test" laid the groundwork for evaluating machine intelligence. The Dartmouth Workshop (1956) is wid

# Analysis Prompt Balance

In [4]:
analysis_prompt = PromptTemplate(
    input_variables=["detailed_response", "concise_response"],
    template="""Compare the following two responses on artificial intelligence:

Detailed response:
{detailed_response}

Concise response:
{concise_response}

Analyze the differences in terms of:
1. Information coverage
2. Clarity and focus
3. Potential use cases for each type of response

Then, suggest strategies for balancing detail and conciseness in prompts."""
)

detailed_response = llm.invoke(detailed_prompt.format(topic=topic)).content
concise_response = llm.invoke(concise_prompt.format(topic=topic)).content

analysis = llm.invoke(analysis_prompt.format(
    detailed_response=detailed_response,
    concise_response=concise_response
)).content

print(analysis)

Let's analyze the differences between the two responses:

**1. Information Coverage:**

* **Detailed Response:** Offers a comprehensive overview, covering the definition, historical context, key components (ML, DL, NLP, Computer Vision, Robotics, Knowledge Representation), practical applications across multiple industries, controversies and debates (job displacement, bias, privacy, autonomous weapons, explainability), and potential future developments (AGI, XAI, AI safety, human-AI collaboration, edge AI, quantum AI).  It's encyclopedic in its scope.

* **Concise Response:** Provides a basic definition and focuses primarily on the importance and broad applications of AI.  It lacks historical context, detailed explanation of AI components, ethical considerations, and future trends.  It's a high-level summary.


**2. Clarity and Focus:**

* **Detailed Response:** While comprehensive, the clarity could be improved with better structuring and use of headings/subheadings.  The sheer amount 

# Strategies for Handeling Long Contexts

1. Chunking- It involves breaking long texts into smaller

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# [A long passage about artificial intelligence, its history, applications, and future prospects...]

long_text = """
Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the 1950s, with key milestones such as the development of the first neural networks and expert systems.
AI encompasses a wide range of subfields, including machine learning, natural language processing, computer vision, and robotics.
Practical applications of AI include speech recognition, image classification, autonomous vehicles, and medical diagnosis.
AI has the potential to revolutionize many industries, from healthcare and finance to transportation and entertainment.
However, there are ongoing debates and controversies surrounding AI, such as concerns about job displacement, bias in algorithms, and the ethical implications of autonomous systems.
Looking ahead, the future of AI holds promise for advancements in areas like explainable AI, AI ethics, and human-AI collaboration.
The intersection of AI with other technologies like blockchain, quantum computing, and biotechnology will likely shape the future of the field.
But as AI continues to evolve, it is essential to consider the societal impact and ethical implications of these technologies.
One of the key challenges for AI researchers and developers is to strike a balance between innovation and responsibility, ensuring that AI benefits society as
a whole while minimizing potential risks.
If managed effectively, AI has the potential to transform our world in ways we can only begin to imagine.
Though the future of AI is uncertain, one thing is clear: the impact of artificial intelligence will be profound and far-reaching.
"""

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Split the text into chunks
chunks = text_splitter.split_text(long_text)

print(f"Number of chunks: {len(chunks)}")
print(f"First chunk: {chunks[0][:200]}...")

Number of chunks: 2
First chunk: Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the...


2. Summarization- Summarization used to condense long texts to retrieve key information

In [7]:
from langchain.docstore.document import Document
from langchain.chains import load_summarize_chain

# Convert text chunks to Document objects
doc_chunks = [Document(page_content=chunk) for chunk in chunks]

# Load the summarization chain
chain = load_summarize_chain(llm, chain_type="map_reduce")

# Summarize the long text
summary_result = chain.invoke(doc_chunks)

print("Summary:")
print(summary_result['output_text'])

Summary:
AI's rapid advancement since the 1950s has revolutionized industries but raises ethical concerns about job displacement and bias.  The future of AI hinges on responsible innovation, focusing on explainability, ethics, and human-AI collaboration to maximize benefits while mitigating risks.


3. Iterative Processing-
For complex tak we use iterative processing.

In [8]:
def iterative_analysis(text, steps):
    """
    Perform iterative analysis on a given text.

    Args:
    text (str): The text to analyze.
    steps (list): List of analysis steps to perform.

    Returns:
    str: The final analysis result.
    """
    result = text
    for step in steps:
        prompt = PromptTemplate(
            input_variables=["text"],
            template=f"Analyze the following text. {step}\n\nText: {{text}}\n\nAnalysis:"
        )
        result = llm.invoke(prompt.format(text=result)).content
    return result

analysis_steps = [
    "Identify the main topics discussed.",
    "Summarize the key points for each topic.",
    "Provide a brief conclusion based on the analysis."
]

final_analysis = iterative_analysis(long_text, analysis_steps)
print("Final Analysis:")
print(final_analysis)

Final Analysis:
The analysis shows the text provides a balanced and comprehensive overview of Artificial Intelligence.  It successfully integrates ethical considerations throughout its discussion of AI's history, current applications, future potential, and societal impact, highlighting the crucial need for responsible development and deployment.  The conclusion accurately reflects this balanced perspective.


In [9]:
tips_prompt = """
Based on the examples and strategies we've explored for managing prompt length and complexity,
provide a list of 5 practical tips for developers working with large language models.
Each tip should be concise and actionable.
"""

tips = llm.invoke(tips_prompt).content
print(tips)

1. **Chunking:** Break down complex tasks into smaller, manageable prompts.

2. **Prioritization:** Focus prompts on the most crucial information; omit unnecessary details.

3. **Iteration:** Refine prompts based on initial model responses; iterate for improved results.

4. **Structured Input:** Use consistent formatting (e.g., JSON) for structured data input.

5. **Contextual Awareness:** Provide sufficient but concise context within the prompt to guide the model.
